In [1]:
import numpy as np

In [2]:
MAJ = ['A', 'B',]

def rm(var):
    allvars = var.split()
    # begining
    allvars[0] = allvars[0].replace('"', '')
    if '{'==allvars[0][0]: 
        allvars[0] = allvars[0][1:]
    # end
    allvars[-1] = allvars[-1].replace('"', '').replace(',', '')
    if '}'==allvars[-1][-1]: 
        allvars[-1] = allvars[-1][:-1]
    # return
    var = ' '.join(allvars)
    
    symbols = {"'":"\\'", 
               'Ö':'{\\"{O}}',
               'ö':'{\\"o}',
               'ü':'{\\"u}',
               '{ extquotesingle}':'{\\textquotesingle}',
               '{ extemdash}':'{\\textemdash}'}
    for key in symbols.keys():
        if key in var:
            var = var.replace(key, symbols[key])
    return var

def format_name(line):
    authors = ''
    unformated_authors = rm(line[1]).split(' and ')
    for author_nbr, author in enumerate(unformated_authors):
        # identify firs and last name
        if ',' in author:
            last_name, first_names = author.split(', ')
        else:
            names = author.split(' ')
            while '' in names: 
                names.remove('')
            last_name = ''
            for i, name in enumerate(names):
                if name[0].islower():
                    last_name = ' '.join(names[i:])
                    break
                elif name=='Di':
                    last_name = ' '.join(names[i:])
                    break
            if last_name=='': 
                if author.split(' ')[-1] == '':
                    last_name = author.split(' ')[-2]
                else:
                    last_name = author.split(' ')[-1]
            first_names = author.split(last_name)[0]
        
        # save last name
        authors += last_name + ', '
        
        # format first name
        formated_first_names = ''
        for k, first_name in enumerate(first_names.split(' ')):
            if first_name != '':
                for i, sub_first_name in enumerate(first_name.split('-')):
                    if sub_first_name != '':
                        # save first letter
                        if sub_first_name[0]=='{':
                            nbropen = 0
                            nbrclose = 0
                            for character in sub_first_name:
                                formated_first_names += character
                                if character=='{': nbropen += 1
                                if character=='}': nbrclose += 1
                                if nbropen==nbrclose: break
                            formated_first_names += '.'
                        else:
                            formated_first_names += sub_first_name[0]+'.'
                        if i<len(first_name.split('-'))-1:
                            formated_first_names +='-'
                if k<len(first_names.split(' '))-1:
                    formated_first_names += ' '
        # save first names
        authors += formated_first_names
        
        # prep for next author
        if author_nbr < len(unformated_authors)-1:
            authors += ' and '
            
        # first authors in label
        if author_nbr == 0:
            label = formated_first_names.replace(' ', '')+last_name.replace(' ', '_')
        elif author_nbr == 1:
            if len(unformated_authors) == 2:
                label += '_'+formated_first_names.replace(' ', '')+last_name.replace(' ', '_')
            else:
                label += '_etal'
    # ! attention, je ne change pas les accents
    return authors, label

def format_citation(citation, ctype='article'):
    
    title, journal = '', ''
    vol, nbr, pages,  = '', '', ''
    month, year = '', ''
    note = ''
    publisher, school = '', ''
    doi, arXivnbr = '', ''
    label = ''
    
    elements = citation.split('\n')
    allkeys = []
    for el in elements:
        line = el.split('=')
        key = line[0].replace(' ', '')
        allkeys += [key]
        if key=='title':
            title = rm(line[1])
        elif key=='author':
            authors, authorlabel = format_name(line)
            if label=='':
                label = authorlabel
            else:
                label = authorlabel+'_'+label
        elif key=='journal':
            journal = rm(line[1])
            if journal=='Class. Quant. Grav.':
                journal = '\cqg'
            elif journal=='Phys. Rev. Lett.':
                journal = '\prl'
            elif journal=='Phys. Rev. D':
                journal = '\prd'
            elif journal=='Phys. Rept.':
                journal = 'Physics Reports'
            elif journal=='JCAP':
                journal = '\jcap'
            elif journal=='Prog. Theor. Phys.':
                journal = 'Progress of Theoretical Physics'
            elif journal=='Mon. Not. Roy. Astron. Soc.':
                journal = '\mnras'
        elif key=='volume':
            vol = rm(line[1])
        elif key=='number' or key=='issue':
            nbr = rm(line[1])
        elif key=='pages':
            pages = rm(line[1])
            pages = pages.replace('--', '-')
        elif key=='month':
            month = rm(line[1])
            months = {'1':['1', 'Jan', 'jan'], 
                      '2':['2', 'Feb', 'feb'], 
                      '3':['3', 'Mar', 'mar'], 
                      '4':['4', 'Apr', 'apr'], 
                      '5':['5', 'May', 'may'], 
                      '6':['6', 'Jun', 'jun'], 
                      '7':['7', 'Jul', 'jul'], 
                      '8':['8', 'Aug', 'aug'], 
                      '9':['9', 'Sep', 'sep'], 
                      '10':['10', 'Oct', 'oct'], 
                      '11':['11', 'Nov', 'nov'], 
                      '12':['12', 'Dec', 'dec']}
            for i in months.keys():
                if month in months[i]:
                    month = i
                    break
        elif key=='year':
            year = rm(line[1])
            if label=='':
                label = year
            else:
                label += '_'+year
        elif key=='doi':
            doi = rm(line[1])
        elif key=='school':
            school = rm(line[1])
        elif key=='note':
            note = rm(line[1])
        elif key=='eprint':
            arXivnbr = rm(line[1])
        elif key=='primaryClass':
            if rm(line[1])=='gr-qc':
                arXivnbr = 'gr-qc/'+arXivnbr
                
    if ctype=='article':
        formated_citation = '@article{'+label+',\n'
        formated_citation += '    title     = {'+title+'},\n'
        formated_citation += '    author    = {'+authors+'},\n'
        formated_citation += '    journal   = {'+journal+'},\n'
        formated_citation += '    volume    = {'+vol+'},\n'
        formated_citation += '    number    = {'+nbr+'},\n'
        formated_citation += '    pages     = {'+pages+'},\n'
        formated_citation += '    month     = {'+month+'},\n'
        formated_citation += '    year      = {'+year+'},\n'
        formated_citation += '    doi       = {'+doi+'},\n'
        if 'note' in allkeys:
            formated_citation += '    note      = {'+note+'},\n'
        formated_citation += '    archivePrefix = {arXiv},\n'
        formated_citation += '    eprint    = {'+arXivnbr+'}}'
    if ctype=='misc':
        formated_citation = '@misc{'+label+',\n'
        formated_citation += '    title     = {'+title+'},\n'
        formated_citation += '    author    = {'+authors+'},\n'
        formated_citation += '    month     = {'+month+'},\n'
        formated_citation += '    year      = {'+year+'},\n'
        if 'note' in allkeys:
            formated_citation += '    note      = {'+note+'},\n'
        formated_citation += '    archivePrefix = {arXiv},\n'
        formated_citation += '    eprint    = {'+arXivnbr+'}}'
    elif 'phd' in ctype:
        formated_citation = '@phdthesis{'+label+',\n'
        formated_citation += '    title     = {'+title+'},\n'
        formated_citation += '    author    = {'+authors+'},\n'
        formated_citation += '    school    = {'+school+'},\n'
        formated_citation += '    year      = {'+year+'},\n'
        if 'note' in allkeys:
            formated_citation += '    note      = {'+note+'},\n'
        formated_citation += '    archivePrefix = {arXiv},\n'
        formated_citation += '    eprint    = {'+arXivnbr+'}}'
    print(formated_citation)

In [3]:
citation = """
@article{Boyle_2019,
doi = {10.1088/1361-6382/ab34e2},
url = {https://dx.doi.org/10.1088/1361-6382/ab34e2},
year = {2019},
month = {sep},
publisher = {IOP Publishing},
volume = {36},
number = {19},
pages = {195006},
author = {Michael Boyle and Daniel Hemberger and Dante A B Iozzo and Geoffrey Lovelace and Serguei Ossokine and Harald P Pfeiffer and Mark A Scheel and Leo C Stein and Charles J Woodford and Aaron B Zimmerman and Nousha Afshari and Kevin Barkett and Jonathan Blackman and Katerina Chatziioannou and Tony Chu and Nicholas Demos and Nils Deppe and Scott E Field and Nils L Fischer and Evan Foley and Heather Fong and Alyssa Garcia and Matthew Giesler and Francois Hebert and Ian Hinder and Reza Katebi and Haroon Khan and Lawrence E Kidder and Prayush Kumar and Kevin Kuper and Halston Lim and Maria Okounkova and Teresita Ramirez and Samuel Rodriguez and Hannes R Rüter and Patricia Schmidt and Bela Szilagyi and Saul A Teukolsky and Vijay Varma and Marissa Walker},
title = {The SXS collaboration catalog of binary black hole simulations},
journal = {Classical and Quantum Gravity},
"""
format_citation(citation)
print()
format_citation(citation, ctype='phd')
print()
format_citation(citation, ctype='misc')

@article{M.Boyle_etal_2019,
    title     = {The SXS collaboration catalog of binary black hole simulations},
    author    = {Boyle, M.  and Hemberger, D.  and Iozzo, D. A. B.  and Lovelace, G.  and Ossokine, S.  and Pfeiffer, H. P.  and Scheel, M. A.  and Stein, L. C.  and Woodford, C. J.  and Zimmerman, A. B.  and Afshari, N.  and Barkett, K.  and Blackman, J.  and Chatziioannou, K.  and Chu, T.  and Demos, N.  and Deppe, N.  and Field, S. E.  and Fischer, N. L.  and Foley, E.  and Fong, H.  and Garcia, A.  and Giesler, M.  and Hebert, F.  and Hinder, I.  and Katebi, R.  and Khan, H.  and Kidder, L. E.  and Kumar, P.  and Kuper, K.  and Lim, H.  and Okounkova, M.  and Ramirez, T.  and Rodriguez, S.  and R{\"u}ter, H. R.  and Schmidt, P.  and Szilagyi, B.  and Teukolsky, S. A.  and Varma, V.  and Walker, M. },
    journal   = {Classical and Quantum Gravity},
    volume    = {36},
    number    = {19},
    pages     = {195006},
    month     = {9},
    year      = {2019},
    doi     

In [12]:
fac = 1
for i in range(1,7):
    fac *= i
    print(i, fac)

1 1
2 2
3 6
4 24
5 120
6 720


In [ ]:
@article{,
         title     = {},
         author    = {},
         journal   = {},
         volume    = {},
         number    = {},
         pages     = {},
         month     = {},
         year      = {},
         doi       = {},
         note      = {},
         archivePrefix = {arXiv},
         eprint    = {}}